In [ ]:
# syft absolute
import syft as sy

In [ ]:
high_client = sy.login(
    url="http://localhost:8080", email="info@openmined.org", password="changethis"
)

In [ ]:
request = high_client.requests[0]

In [ ]:
request

In [ ]:
request.code

In [ ]:
# syft absolute
from syft.service.code.user_code import UserCode
from syft.service.request.request import Request


def execute_request(client_high, request) -> dict:
    if not isinstance(request, Request):
        return "This is not a request"

    code = request.code
    if not isinstance(code, UserCode):
        return "No usercode found"

    func_name = request.code.service_func_name
    api_func = getattr(client_high.code, func_name, None)
    if api_func is None:
        return "Code name was not found on the client."

    job = api_func(blocking=False)
    return job

In [ ]:
job = execute_request(high_client, request)

In [ ]:
job

In [ ]:
job.wait()

In [ ]:
job_info = job.info(result=True)
request.deposit_result(job_info, approve=True)

In [ ]:
# syft absolute


def approve_and_deposit(do_client, request_id):
    request = do_client.requests.get_by_uid(uid=sy.UID(request_id))

    if not isinstance(request, Request):
        return "This is not a request"

    code = request.code
    if not isinstance(code, UserCode):
        return "No usercode found on request"

    func_name = request.code.service_func_name
    api_func = getattr(do_client.code, func_name, None)
    if api_func is None:
        return "Code name not found in the client's code"

    job = api_func(blocking=False)
    job.wait()

    job_info = job.info(result=True)
    return request.deposit_result(job_info, approve=True)

In [ ]:
def list_pending(do_client):
    for request in do_client.requests:
        if str(request.status) == "RequestStatus.PENDING":
            print(request.id)
            print(request.code.constants["query"].val)
            print()

In [ ]:
list_pending(high_client)

In [ ]:
approve_and_deposit(high_client, "5c0e75dc293b43c3a16b138a58232408")

In [ ]:
approve_and_deposit(high_client, "936b2a5e8d2f4106980650c6214aa23d")